In [ ]:
!pip install pandas scikit-learn tensorflow openpyxl

In [ ]:
import pandas as pd
from google.colab import files

# Function to process the uploaded file
def process_file(uploaded_file):
    # Read the Excel file
    df = pd.read_excel(uploaded_file)

    # Keep only the specified columns
    df = df[['Occupation', 'Importance']]

    # Extract the file name without extension
    file_stem = uploaded_file.rsplit('.', 1)[0]

    # Rename the 'Importance' column
    df.loc[:, 'Importance'] = df['Importance'].copy()  # Copy the column to avoid SettingWithCopyWarning
    df.rename(columns={'Importance': file_stem}, inplace=True)

    # Save the modified DataFrame to a new Excel file
    output_filename = "processed_" + uploaded_file
    df.to_excel(output_filename, index=False)

    return output_filename

# Upload files
uploaded = files.upload()

# Process each uploaded file
for filename in uploaded.keys():
    output_filename = process_file(filename)
    print(f"Processed file saved as: {output_filename}")


Saving Extent_Flexibility.xlsx to Extent_Flexibility.xlsx
Saving Stamina.xlsx to Stamina.xlsx
Saving Fluency_of_Ideas.xlsx to Fluency_of_Ideas.xlsx
Saving Active_Learning.xlsx to Active_Learning.xlsx
Saving Active_Listening.xlsx to Active_Listening.xlsx
Saving Biology.xlsx to Biology.xlsx
Saving Chemistry.xlsx to Chemistry.xlsx
Saving Computers_and_Electronics.xlsx to Computers_and_Electronics.xlsx
Saving Critical_Thinking.xlsx to Critical_Thinking.xlsx
Saving Design.xlsx to Design.xlsx
Saving Economics_and_Accounting.xlsx to Economics_and_Accounting.xlsx
Saving English_Language.xlsx to English_Language.xlsx
Saving Foreign_Language.xlsx to Foreign_Language.xlsx
Saving Geography.xlsx to Geography.xlsx
Saving Law_and_Government.xlsx to Law_and_Government.xlsx
Saving Monitoring.xlsx to Monitoring.xlsx
Saving Memorization.xlsx to Memorization.xlsx
Saving Medicine_and_Dentistry.xlsx to Medicine_and_Dentistry.xlsx
Saving Mathematics.xlsx to Mathematics.xlsx
Saving Mathematical_Reasoning.xlsx

<ipython-input-10-b9fe8ca59a7e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Importance': file_stem}, inplace=True)


Processed file saved as: processed_Critical_Thinking.xlsx
Processed file saved as: processed_Design.xlsx
Processed file saved as: processed_Economics_and_Accounting.xlsx
Processed file saved as: processed_English_Language.xlsx


<ipython-input-10-b9fe8ca59a7e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Importance': file_stem}, inplace=True)


Processed file saved as: processed_Foreign_Language.xlsx
Processed file saved as: processed_Geography.xlsx
Processed file saved as: processed_Law_and_Government.xlsx
Processed file saved as: processed_Monitoring.xlsx
Processed file saved as: processed_Memorization.xlsx
Processed file saved as: processed_Medicine_and_Dentistry.xlsx
Processed file saved as: processed_Mathematics.xlsx
Processed file saved as: processed_Mathematical_Reasoning.xlsx
Processed file saved as: processed_Oral_Expression.xlsx
Processed file saved as: processed_Time_Sharing.xlsx
Processed file saved as: processed_Telecommunications.xlsx
Processed file saved as: processed_Static_Strength.xlsx
Processed file saved as: processed_Selective_Attention.xlsx


<ipython-input-10-b9fe8ca59a7e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Importance': file_stem}, inplace=True)


Processed file saved as: processed_Psychology.xlsx
Processed file saved as: processed_Physics.xlsx


In [ ]:
import pandas as pd
import os
from glob import glob

# List all processed files in the directory
processed_files = glob("processed_*.xlsx")

# Initialize an empty list to hold the processed DataFrames
dfs = []

# Read each processed file and add to the list of DataFrames
for file in processed_files:
    df = pd.read_excel(file)
    dfs.append(df)

# Merge all DataFrames on the 'Occupation' column
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on='Occupation', how='outer')

# Save the merged DataFrame to a new Excel file
merged_df.to_excel("merged_processed_files.xlsx", index=False)

print("Merged file saved as: merged_processed_files.xlsx")


Merged file saved as: merged_processed_files.xlsx


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Load the dataset
file_path = '/content/final_copy.xlsx'  # Update this with the actual path
df = pd.read_excel(file_path)

# Encode the 'Job Family' column
label_encoder = LabelEncoder()
df['Job Family'] = label_encoder.fit_transform(df['Job Family'])

# Separate features and target
X = df.drop(columns=['Occupation', 'Job Family'])
y = df['Job Family']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the callbacks
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[checkpoint, early_stopping])

# Evaluate the model
model.load_weights('best_model.keras')
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Save the model
model.save('final_model.keras')

# Predict top 3 job families for a given input
import numpy as np

def predict_top_3_job_families(input_data):
    input_scaled = scaler.transform([input_data])
    predictions = model.predict(input_scaled)[0]
    top_3_indices = predictions.argsort()[-3:][::-1]
    top_3_job_families = label_encoder.inverse_transform(top_3_indices)
    return top_3_job_families

# Example usage
input_data = X_test[0]
top_3_job_families = predict_top_3_job_families(input_data)
print(f'Top 3 Job Families: {top_3_job_families}')


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.0641 - loss: 4.0840 - val_accuracy: 0.1727 - val_loss: 2.9465
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0698 - loss: 3.8256 - val_accuracy: 0.2878 - val_loss: 2.7830
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1144 - loss: 3.3706 - val_accuracy: 0.3094 - val_loss: 2.6264
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1916 - loss: 3.0219 - val_accuracy: 0.3165 - val_loss: 2.4894
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2150 - loss: 2.9682 - val_accuracy: 0.3453 - val_loss: 2.3633
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2923 - loss: 2.5740 - val_accuracy: 0.3453 - val_loss: 2.2671
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3011 - loss: 2.5449 - val_accuracy: 0.3453 - val_loss: 2.1893
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2852 - loss: 2.4494 - val_accuracy: 0.3

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Top 3 Job Families: ['Office and Administrative Support' 'Production'
 'Personal Care and Service']


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
import numpy as np

# Load the dataset
file_path = '/content/final_copy.xlsx'  # Update this with the actual path
df = pd.read_excel(file_path)

# Encode the 'Job Family' column
label_encoder = LabelEncoder()
df['Job Family'] = label_encoder.fit_transform(df['Job Family'])

# Separate features and target
X = df.drop(columns=['Occupation', 'Job Family'])
y = df['Job Family']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define a function to create the model
def create_model():
    model = Sequential([
        Input(shape=(X.shape[1],)),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * np.exp(-0.1))

# K-Fold Cross-Validation
kf = KFold(n_splits=5)
best_model = None
best_accuracy = 0

for train_index, val_index in kf.split(X_scaled):
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = create_model()

    checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    lr_scheduler = LearningRateScheduler(scheduler)

    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint, early_stopping, lr_scheduler])

    val_loss, val_accuracy = model.evaluate(X_val, y_val)

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model = model

# Evaluate the best model on the test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Save the best model
best_model.save('final_model.keras')

# Predict top 3 job families for a given input
def predict_top_3_job_families(input_data):
    input_scaled = scaler.transform([input_data])
    predictions = best_model.predict(input_scaled)[0]
    top_3_indices = predictions.argsort()[-3:][::-1]
    top_3_job_families = label_encoder.inverse_transform(top_3_indices)
    return top_3_job_families

# Example usage
input_data = X_test[0]
top_3_job_families = predict_top_3_job_families(input_data)
print(f'Top 3 Job Families: {top_3_job_families}')


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - accuracy: 0.0357 - loss: 4.3418 - val_accuracy: 0.1437 - val_loss: 2.9712 - learning_rate: 0.0010
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.1117 - loss: 3.4529 - val_accuracy: 0.2644 - val_loss: 2.7975 - learning_rate: 0.0010
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1983 - loss: 3.0981 - val_accuracy: 0.2816 - val_loss: 2.6403 - learning_rate: 0.0010
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.2367 - loss: 2.8006 - val_accuracy: 0.2816 - val_loss: 2.4936 - learning_rate: 0.0010
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.2635 - loss: 2.5946 - val_accuracy: 0.3276 - val_loss: 2.3559 - learning_rate: 0.0010
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2594 - loss: 2.5395 - val_accuracy: 0.3506 - val_loss: 2.2344 - learning_rate: 0.0010
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3013 - loss: 2.4097 - 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Top 3 Job Families: ['Personal Care and Service' 'Farming, Fishing, and Forestry' 'Production']
